# JuliaのTips集
このページはJuliaでの実装におけるTips (詰まったところの解決策)をまとめたものである。体系的にまとまってはいない。

## 1. 関数名の!記号
単なる**慣習**として関数への入力を変更する場合に!を付ける。

関数内で配列を変更する場合には注意が必要である。以下に入力された配列を同じサイズの要素1の配列で置き換える、ということを目的として書かれた2つの関数がある。違いは`v`の後に`[:]`としているかどうかである。

In [1]:
function wrong!(a::Array)
    a = ones(size(a))
end

function right!(a::Array)
    a[:] = ones(size(a))
end

right! (generic function with 1 method)

実行すると`wrong!`の場合には入力された配列が変更されていないことがわかる。

In [2]:
using Random
v = rand(2, 2)
println("v : ", v)

wrong!(v)
println("wrong : ", v)

right!(v)
println("right : ", v)

v : 

[0.6142967526229162 0.7504305854875333; 0.12024842942806147 0.35030182796528386]
wrong : [0.6142967526229162 0.7504305854875333; 0.12024842942806147 0.35030182796528386]
right : [1.0 1.0; 1.0 1.0]


## 2. 配列の1次元化
配列を一次元化(flatten)する方法。まずは3次元配列を作成する。

In [3]:
B = rand(2, 2, 2)

2×2×2 Array{Float64,3}:
[:, :, 1] =
 0.258224   0.0598795
 0.0355192  0.514187

[:, :, 2] =
 0.139675  0.955327
 0.573969  0.727775

用意されている`flatten`を素直に用いると次のようになる。

In [4]:
import Base.Iterators: flatten
collect(flatten(B))

8-element Array{Float64,1}:
 0.2582244836954313
 0.03551922998167911
 0.05987945348805823
 0.5141870513755218
 0.13967497771778103
 0.5739691086220715
 0.9553266238169829
 0.7277751289460395

ただし、単に`B[:]`とするだけでもよい。

In [5]:
B[:]

8-element Array{Float64,1}:
 0.2582244836954313
 0.03551922998167911
 0.05987945348805823
 0.5141870513755218
 0.13967497771778103
 0.5739691086220715
 0.9553266238169829
 0.7277751289460395

## 3. 行列の行・列ごとの正規化
シミュレーションにおいてニューロン間の重み行列を行あるいは列ごとに正規化 (weight normalization)する場合がある。これは各ニューロンへの入力の大きさを同じにする働きや重みの発散を防ぐ役割がある。以下では行ごとの和を1にする。

In [6]:
W = rand(3,3)

3×3 Array{Float64,2}:
 0.461563  0.0778242  0.295994
 0.981843  0.969748   0.425056
 0.716388  0.553162   0.0773432

In [7]:
Wnormed = W ./ sum(W, dims=1)

3×3 Array{Float64,2}:
 0.213707  0.0486178  0.370737
 0.4546    0.605814   0.532389
 0.331693  0.345568   0.0968737

In [8]:
println(sum(Wnormed, dims=1))

[1.0 1.0 1.0]


## 4. 行列の結合 (concatenate)
行列の結合はMATLABに近い形式で行うことができる。まず、2つの行列A, Bを用意する。

In [9]:
A = [1 2; 3 4]

2×2 Array{Int64,2}:
 1  2
 3  4

In [10]:
B = [4 5 6; 7 8 9]

2×3 Array{Int64,2}:
 4  5  6
 7  8  9

### 4.1 水平結合 (Horizontal concatenation)
`hcat`を使うやり方と、`[ ]`を使うやり方がある。

In [11]:
H1 = hcat(A,B)

2×5 Array{Int64,2}:
 1  2  4  5  6
 3  4  7  8  9

In [12]:
H2 = [A B]

2×5 Array{Int64,2}:
 1  2  4  5  6
 3  4  7  8  9

なお、MATLABのように次のようにすると正しく結合はされない。

In [13]:
H3 = [A, B]

2-element Array{Array{Int64,2},1}:
 [1 2; 3 4]
 [4 5 6; 7 8 9]

### 4.2 垂直結合 (Vertical concatenation)

In [14]:
V1 = vcat(A, B')

5×2 Array{Int64,2}:
 1  2
 3  4
 4  7
 5  8
 6  9

In [15]:
V2 = [A; B']

5×2 Array{Int64,2}:
 1  2
 3  4
 4  7
 5  8
 6  9

In [16]:
[V2 [A;B']]

5×4 Array{Int64,2}:
 1  2  1  2
 3  4  3  4
 4  7  4  7
 5  8  5  8
 6  9  6  9

## 5. 配列に新しい軸を追加
要はnumpyでの`A[None, :]`や`A[np.newaxis, :]`のようなことがしたい場合。やや面倒だが、`reshape`を使うか、`[CartesianIndex()]`を用いる。

In [17]:
v = rand(3)

3-element Array{Float64,1}:
 0.3578091341460987
 0.8696846050820677
 0.16932555721987574

In [18]:
newaxis = [CartesianIndex()]
v1 = v[newaxis, :]

1×3 Array{Float64,2}:
 0.357809  0.869685  0.169326

## 6. Array{Array{Float64, x},1}をArray{Float64, x+1}に変換
numpyでは`array([matrix for i in range()])`などを用いると、1次元配列のリストを2次元配列に変換できた。Juliaでも同様にする場合は`hcat(...)`や`cat(...)`を用いる。

In [19]:
A1 = [i*rand(3) for i=1:5]

println("Type : ", typeof(A1))
println("Size : ", size(A1))

Type : Array{Array{Float64,1},1}
Size : (5,)


In [20]:
A2 = hcat(A1...)'

println("Type : ", typeof(A2))
println("Size : ", size(A2))

Type : LinearAlgebra.Adjoint{Float64,Array{Float64,2}}
Size : (5, 3)


以下は多次元配列の場合。`cat(...)`で配列を結合し、`permitedims`で転置する。

In [21]:
B1 = [i*rand(3, 4, 5) for i=1:6]

println("Type : ", typeof(B1))
println("Size : ", size(B1))

Type : Array{Array{Float64,3},1}
Size : (6,)


In [22]:
B2 = permutedims(cat(B1..., dims=4), [4, 1, 2, 3])

println("Type : ", typeof(B2))
println("Size : ", size(B2))

Type : Array{Float64,4}
Size : (6, 3, 4, 5)


## 7. 二項分布 (bernoulli distribution)のサンプリング

In [23]:
p = 0.7
N = 100

100

In [24]:
using BenchmarkTools

In [25]:
@benchmark floor.(p .+ rand(N))

BenchmarkTools.Trial: 
  memory estimate:  1.91 KiB
  allocs estimate:  7
  --------------
  minimum time:     690.244 ns (0.00% GC)
  median time:      718.896 ns (0.00% GC)
  mean time:        763.599 ns (3.68% GC)
  maximum time:     6.477 μs (87.05% GC)
  --------------
  samples:          10000
  evals/sample:     164

In [26]:
println(sum(floor.(p .+ rand(N)) .== 1.0) / N) 

0.76


In [27]:
@benchmark 1.0f0 * (p .≥ rand(N))

BenchmarkTools.Trial: 
  memory estimate:  1.55 KiB
  allocs estimate:  6
  --------------
  minimum time:     884.615 ns (0.00% GC)
  median time:      925.667 ns (0.00% GC)
  mean time:        987.642 ns (2.61% GC)
  maximum time:     22.236 μs (94.86% GC)
  --------------
  samples:          10000
  evals/sample:     39

In [28]:
println(sum(1.0f0 * (p .≥ rand(N)) .== 1.0) / N) 

0.74
